In [ ]:
import requests
import pandas as pd
import json
import random

In [ ]:
json_response = requests.get('https://api.icowatchlist.com/public/v1/finished').json()
data = json_response['ico']['finished']
with open('icoWatchListData.json', 'w') as f:
    f.write(json.dumps(data))
with open('ico_info_list.json', 'r') as f:
    more_data = json.loads(f.read())

In [ ]:
documents = [item['description'] for item in data] + [item['description'] for item in more_data if 'description' in item]
print(len(documents))
more_documents

In [ ]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
stemmer = SnowballStemmer("english")
np.random.seed(2018)
nltk.download('wordnet')

In [ ]:
def lemmatize_stemming(text):
    lemmentized_text = WordNetLemmatizer().lemmatize(text, pos='v')
    return stemmer.stem(lemmentized_text)
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [ ]:
processed_docs = [preprocess(document) for document in documents]
processed_docs[:10]

In [ ]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

In [ ]:
dictionary.filter_extremes(no_below=3, no_above=1, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_doc_4 = bow_corpus[980]
for i in range(len(bow_doc_4)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4[i][0], dictionary[bow_doc_4[i][0]], bow_doc_4[i][1]))

In [ ]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

## lda with bow

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=5, workers=2)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

## lda with tf-idf

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=5, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {}\n Word: {}\n'.format(idx, topic))

In [ ]:
print(documents[1])
bow_corpus[1]

In [52]:
rand_index = random.randint(0,len(documents))
print('the random document : {}'.format(documents[rand_index]))
for index, score in sorted(lda_model[bow_corpus[rand_index]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))

the random document :  Peer-to-peer lending platform that provides instant cash advances against cryptocurrency collateral.

Score: 0.9181690216064453	 
Topic: 0.041*"platform" + 0.029*"decentr" + 0.022*"power" + 0.022*"blockchain" + 0.020*"cryptocurr" + 0.018*"peer" + 0.016*"token" + 0.014*"base" + 0.014*"social" + 0.011*"network"


In [53]:
for index, score in sorted(lda_model_tfidf[bow_corpus[rand_index]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.4850538969039917	 
Topic: 0.018*"decentr" + 0.018*"platform" + 0.018*"asset" + 0.017*"invest" + 0.016*"token" + 0.016*"peer" + 0.015*"lend" + 0.015*"blockchain" + 0.014*"exchang" + 0.014*"digit"

Score: 0.44220274686813354	 
Topic: 0.018*"blockchain" + 0.017*"power" + 0.014*"decentr" + 0.013*"smart" + 0.013*"network" + 0.013*"platform" + 0.012*"contract" + 0.010*"technolog" + 0.010*"intellig" + 0.010*"marketplac"
